<a href="https://colab.research.google.com/github/weedge/doraemon-nb/blob/main/tts_step_audio_stream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install

In [ ]:
!git clone --recursive https://github.com/weedge/Step-Audio.git -b feat/stream

In [ ]:
%cd /content/Step-Audio

In [ ]:
!git checkout . && git pull origin feat/stream

In [ ]:
!nvcc --version

In [ ]:
!ls /usr/local/cuda-12.5/lib64/libcublasLt*

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
!pip show torch torchaudio torchvision

In [ ]:
#!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1 --index-url https://download.pytorch.org/whl/cu121

In [ ]:
!pip install onnxruntime-gpu==1.20.1

In [ ]:
!pip show onnxruntime-gpu

In [ ]:
!apt install -y sox

In [ ]:
!pip install -q python-dotenv

# download ckpt


In [ ]:
from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

In [ ]:
!huggingface-cli login --token $HF_TOKEN --add-to-git-credential


In [ ]:
!huggingface-cli download stepfun-ai/Step-Audio-Tokenizer --quie --local-dir /content/models/stepfun-ai/Step-Audio-Tokenizer


In [ ]:
!huggingface-cli download stepfun-ai/Step-Audio-TTS-3B --quie --local-dir /content/models/stepfun-ai/Step-Audio-TTS-3B


# tts static batch stream inference

In [ ]:
%cd /content/Step-Audio

In [33]:
# tts without ref audio
!TTS_TEXT="你好！" python tts_inference_stream.py --model-path=/content/models/stepfun-ai/ --synthesis-type=tts --output-path=./ --stream=static_batch --stream-factor=2

If you want to use the speaker diarization, please `pip install hdbscan`
If you want use mossformer, please install rotary_embedding_torch by: 
 pip install -U rotary_embedding_torch
If you want use mossformer, please install rotary_embedding_torch by: 
 pip install -U rotary_embedding_torch
Please Requires the ffmpeg CLI and `ffmpeg-python` package to be installed.
If you want use mossformer, please install rotary_embedding_torch by: 
 pip install -U rotary_embedding_torch
If you want use mossformer, please install rotary_embedding_torch by: 
 pip install -U rotary_embedding_torch

tables: 

-----------    ** dataset_classes **    --------------
| class name   | class location                                                          |
| AudioDataset | /content/Step-Audio/funasr_detach/datasets/audio_datasets/datasets.py:7 |
-----------    ** index_ds_classes **    --------------
| class name            | class location                                                           |
| Inde

In [ ]:
# tts without ref audio
!python tts_inference_stream.py --model-path=/content/models/stepfun-ai/ --synthesis-type=tts --output-path=./ --stream=static_batch --stream-factor=4

In [34]:
from IPython.display import Audio
Audio('./output_tts_stream.wav',autoplay=True)

In [ ]:
from IPython.display import Audio
Audio('examples/prompt_wav_yuqian.wav',autoplay=True)

In [ ]:
# tts with ref audio
!python tts_inference_stream.py --model-path=/content/models/stepfun-ai/ --synthesis-type=clone --output-path=./ --stream=static_batch --stream-factor=4

In [ ]:
# tts with ref audio
!TTS_TEXT="你好！" python tts_inference_stream.py --model-path=/content/models/stepfun-ai/ --synthesis-type=clone --output-path=./ --stream=static_batch --stream-factor=2

In [ ]:
from IPython.display import Audio
Audio('./output_clone_stream.wav',autoplay=True)

# text to speech

In [ ]:
%cd /content/Step-Audio

In [ ]:
# tts without ref audio
!python tts_inference.py --model-path=/content/models/stepfun-ai/ --synthesis-type=tts --output-path=./

In [ ]:
from IPython.display import Audio
Audio('./output_tts.wav',autoplay=True)

In [ ]:
from IPython.display import Audio
Audio('examples/prompt_wav_yuqian.wav',autoplay=True)

In [ ]:
# tts with ref audio
!python tts_inference.py --model-path=/content/models/stepfun-ai/ --synthesis-type=clone --output-path=./

In [ ]:
from IPython.display import Audio
Audio('./output_clone.wav',autoplay=True)

# step1 LM tokenizer

In [ ]:
%cd /content/Step-Audio

In [ ]:
import os
import sys
sys.path.insert(1, "/content/Step-Audio")

In [ ]:
from tts import StepAudioTTS
from tokenizer import StepAudioTokenizer

encoder = StepAudioTokenizer(f"/content/models/stepfun-ai/Step-Audio-Tokenizer")
print(encoder)
tts_engine = StepAudioTTS(f"/content/models/stepfun-ai/Step-Audio-TTS-3B", encoder)
print(tts_engine)


In [ ]:
text = "本文介绍了Step-Audio，这是一个开源的智能语音交互框架，旨在解决现有开源模型在语音数据采集成本高、动态控制能力弱和智能水平有限等问题。"
text = "君不见黄河之水天上来，奔流到海不复回。君不见高堂明镜悲白发，朝如青丝暮成雪。人生得意须尽欢，莫使金樽空对月。天生我材必有用，千金散尽还复来。"
#instruction_name = tts_engine.detect_instruction_name(text)
prompt_speaker_info = tts_engine.speakers_info["Tingting"]
print(prompt_speaker_info)

In [ ]:
token_ids = tts_engine.tokenize(text,prompt_speaker_info["prompt_text"],"",prompt_speaker_info["prompt_code"])
print(token_ids)

In [ ]:
text = tts_engine.tokenizer.decode(token_ids)
print(text)

# step1 LM generate

## generate

In [ ]:
import torch

from tts import LogitsProcessorList,RepetitionAwareLogitsProcessor

output_ids = tts_engine.llm.generate(
  torch.tensor([token_ids]).to(torch.long).to("cuda"),
  max_length=8192,
  temperature=0.7,
  do_sample=True,
  logits_processor=LogitsProcessorList([RepetitionAwareLogitsProcessor()]),
)
print(output_ids)

In [ ]:
text = tts_engine.tokenizer.decode(output_ids[0])
print(text)

In [ ]:
audio_token_ids = output_ids[0][len(token_ids):-1] # skip <|EOT|> token
print(audio_token_ids)
audio_token_text = tts_engine.tokenizer.decode(audio_token_ids)
print(audio_token_text)

## stream generate

In [ ]:
from queue import Queue

from transformers.generation.streamers import BaseStreamer

class TokenStreamer(BaseStreamer):
    def __init__(self, skip_prompt: bool = False, timeout=None):
        self.skip_prompt = skip_prompt

        # variables used in the streaming process
        self.token_queue = Queue()
        self.stop_signal = None
        self.next_tokens_are_prompt = True
        self.timeout = timeout

    def put(self, value):
        if len(value.shape) > 1 and value.shape[0] > 1:
            raise ValueError("TextStreamer only supports batch size 1")
        elif len(value.shape) > 1:
            value = value[0]
        #print(value)

        if self.skip_prompt and self.next_tokens_are_prompt:
            self.next_tokens_are_prompt = False
            return

        for token in value.tolist():
            self.token_queue.put(token)

    def end(self):
        self.token_queue.put(self.stop_signal)

    def __iter__(self):
        return self

    def __next__(self):
        value = self.token_queue.get(timeout=self.timeout)
        if value == self.stop_signal:
            raise StopIteration()
        else:
            return value

In [ ]:
streamer = TokenStreamer(skip_prompt=True)
print(streamer)

In [ ]:
from threading import Thread

import torch

from tts import LogitsProcessorList,RepetitionAwareLogitsProcessor

generation_kwargs = dict(
  input_ids=torch.tensor([token_ids]).to(torch.long).to("cuda"),
  eos_token_id=3,
  streamer=streamer,
  max_length=8192,
  temperature=0.7,
  do_sample=True,
  logits_processor=LogitsProcessorList([RepetitionAwareLogitsProcessor()]),
)
print("generation_kwargs", generation_kwargs)

thread = Thread(target=tts_engine.llm.generate, kwargs=generation_kwargs)
thread.start()

generated_ids = []
for token_id in streamer:
  print(token_id, end=",", flush=True)
  generated_ids.append(token_id)

print("\n",generated_ids)

In [ ]:
audio_token_text = tts_engine.tokenizer.decode(generated_ids)
print(audio_token_text)

In [ ]:
prompt_token_text = tts_engine.tokenizer.decode(token_ids)
print(prompt_token_text)

In [ ]:
audio_token_text = tts_engine.tokenizer.decode(3)
print(audio_token_text)

In [ ]:
audio_token_ids = generated_ids[:-1] # skip <|EOT|> token
print(audio_token_ids)
audio_token_text = tts_engine.tokenizer.decode(audio_token_ids)
print(audio_token_text)

In [ ]:
from threading import Thread
import math

import torch

from tts import LogitsProcessorList,RepetitionAwareLogitsProcessor

generation_kwargs = dict(
  input_ids=torch.tensor([token_ids]).to(torch.long).to("cuda"),
  eos_token_id=3,
  streamer=streamer,
  max_length=8192,
  temperature=0.7,
  do_sample=True,
  logits_processor=LogitsProcessorList([RepetitionAwareLogitsProcessor()]),
)
print("generation_kwargs", generation_kwargs)

thread = Thread(target=tts_engine.llm.generate, kwargs=generation_kwargs)
thread.start()

#batch_size = 30
stream_factor=2 # >=2 increase for better speech quality, but rft slow (speech quality vs rft slow)
batch_size = math.ceil(stream_factor*tts_engine.common_cosy_model.model.flow.input_frame_rate)
print(batch_size)

generated_ids = []
for token_id in streamer:
  #print(token_id, end=",", flush=True)
  if token_id == 3:  # skip <|EOT|>, break
    break
  generated_ids.append(token_id)
  if len(generated_ids) % batch_size == 0:
    print("\n",generated_ids)
    batch = (
        torch.tensor(generated_ids)
        .unsqueeze(0)
        .to(tts_engine.common_cosy_model.model.device)
    )  # [T] -> [1,T]
    print(batch,batch.shape)
    # Process each batch
    sub_tts_speech = tts_engine.common_cosy_model.token_to_wav_offline(
                    batch,
                    prompt_speaker_info["cosy_speech_feat"].to(torch.bfloat16),
                    prompt_speaker_info["cosy_speech_feat_len"],
                    prompt_speaker_info["cosy_prompt_token"],
                    prompt_speaker_info["cosy_prompt_token_len"],
                    prompt_speaker_info["cosy_speech_embedding"].to(torch.bfloat16),
    )
    print(sub_tts_speech,sub_tts_speech.shape)



print("\n",generated_ids)


# generate waveform from audio vq codes

In [ ]:
print(prompt_speaker_info)

In [ ]:
prompt_speaker_info["cosy_speech_feat"]

In [ ]:
prompt_speaker_info["cosy_speech_feat_len"]

In [ ]:
prompt_speaker_info["cosy_prompt_token"].to("cuda")

In [ ]:
prompt_speaker_info["cosy_prompt_token_len"].to("cuda")

In [ ]:
prompt_speaker_info["cosy_speech_embedding"]

In [ ]:
# 你好
audio_token_ids = [66424, 65801, 66725, 66575, 68722, 65949, 66106, 67604, 69789, 69810, 65589, 65589, 69810, 68289, 68295, 66027, 66547, 68649, 68295, 67783, 66107, 66547, 67783, 67909, 67909, 65613, 65613, 66586, 66586, 70493]
#audio_token_text = "<audio_888><audio_265><audio_1189><audio_1039><audio_3186><audio_413><audio_570><audio_2068><audio_4253><audio_4274><audio_53><audio_53><audio_4274><audio_2753><audio_2759><audio_491><audio_1011><audio_3113><audio_2759><audio_2247><audio_571><audio_1011><audio_2247><audio_2373><audio_2373><audio_77><audio_77><audio_1050><audio_1050><audio_4957>"

In [ ]:
# 本文介绍了Step-Audio，这是一个开源的智能语音交互框架，旨在解决现有开源模型在语音数据采集成本高、动态控制能力弱和智能水平有限等问题。
audio_token_ids=[
        66077, 65712, 66725, 68670, 66614, 65949, 66185, 66609, 67812, 68014,
        66185, 65607, 68130, 69827, 68810, 65607, 66486, 68076, 68813, 67130,
        65645, 65972, 67597, 69622, 68175, 65767, 66486, 68803, 66903, 69492,
        65557, 65557, 66885, 68443, 68139, 65754, 65977, 69097, 67816, 67816,
        65621, 66547, 67816, 67778, 67587, 65613, 65840, 68294, 70458, 70458,
        66471, 66361, 70458, 67445, 67526, 66029, 66029, 70476, 69264, 68622,
        66381, 66133, 67460, 70160, 68708, 66177, 66217, 66669, 67075, 66679,
        66041, 66041, 67021, 67260, 66762, 66447, 66486, 68272, 66885, 66885,
        66502, 66502, 67300, 67878, 70160, 66502, 66309, 68466, 66629, 68200,
        66284, 65811, 66992, 66992, 68073, 66167, 65977, 68073, 66940, 68064,
        66167, 66061, 68064, 67250, 67250, 65840, 66283, 67587, 68006, 68064,
        65684, 65684, 68029, 68175, 68803, 65682, 66035, 66594, 66601, 66588,
        65795, 65795, 68258, 66619, 66619, 66279, 66486, 70112, 70576, 66561,
        66279, 65911, 67092, 68610, 70576, 65911, 66237, 67966, 66580, 67753,
        65735, 65735, 67130, 70321, 67277, 65735, 66254, 67049, 69206, 67333,
        66336, 65821, 67333, 69559, 67633, 65897, 65897, 66592, 67963, 66907,
        65973, 66215, 67129, 68959, 70153, 66215, 66472, 67333, 68130, 68115,
        66241, 66241, 66946, 68128, 67945, 66402, 66536, 67945, 69666, 66600,
        65549, 65829, 69181, 69346, 68810, 65847, 66472, 66829, 68813, 68806,
        66366, 66366, 69492, 67328, 66794, 66402, 66429, 70507, 66633, 68815,
        66443, 66402, 70052, 66794, 66885, 65554, 66048, 66946, 67801, 70114,
        65792, 65792, 68800, 66679, 69418, 66167, 66012, 66568, 66568, 67587,
        65967, 66167, 68064, 68064, 67250, 65840, 65840, 70458, 70458, 66613,
        66132, 65705, 70209, 70209, 67526, 65617, 65750, 66605, 67129, 66566,
        66040, 66040, 66566, 68076, 66661, 66360, 66315, 66981, 66679, 66593,
        65947, 65947, 68076, 68813, 67913, 65972, 65972, 67851, 66829, 67352,
        65972, 66335, 67945, 68200, 67049, 66379, 66379, 67736, 68800, 67206,
        65877, 65877, 69404, 66753, 67261, 65899, 65899, 67130, 67604, 69401,
        65899, 65911, 66992, 68356, 69524, 65911, 66472, 68847, 67966, 67572,
        65735, 65735, 67658, 70321, 66592, 66160, 66472, 68038, 66569, 68989,
        66160, 65969, 67362, 68169, 68803, 66421, 66472, 67184, 69404, 66753,
        66466, 66290, 68220, 69980, 68747, 66167, 65840, 69624, 66682, 68768,
        66167, 66471, 67526, 70458, 66613, 65840, 66283, 70209, 69538, 68076,
        66102, 66360, 68254, 69559, 66981, 66402, 66035, 66669, 70616, 66619,
        66241, 65549, 67945, 67945, 69666, 65549, 66472, 66600, 66619, 69181,
        66335, 65970, 68128, 70639, 69798, 65970, 66472, 69798, 67318, 66644,
        66137, 66137, 67502, 67054, 66935, 66402, 65615, 67277, 68364, 66829,
        66538, 66538, 68254, 67966, 66580, 66402, 65554, 67071, 70616, 67803,
        65964, 66335, 66889, 68220, 67877, 66423, 66472, 70328, 70209, 68624,
        66127, 66127, 66881, 67440, 67314, 66402, 65669, 67633, 69436, 68248,
        65987, 65987, 70576, 66614, 66918, 66402, 65977, 66918, 67783, 67070,
        65967, 66167, 67070, 66940, 68867, 65545, 65840, 68064, 68064, 67250,
        66471, 65705, 67250, 68064, 67424, 65752, 65750, 69897, 67873, 66765,
        66539, 65932, 67128, 68069, 68663, 65932, 66486, 70160, 67966, 66580,
        66335, 66443, 69834, 70474, 66962, 66443, 66237, 67442, 66794, 68069,
        65938, 65938, 67935, 70328, 66588, 66215, 66486, 70153, 67764, 68847,
        66215, 65968, 66618, 69853, 66663, 66257, 65668, 67906, 70321, 67502,
        65714, 65714, 68127, 69825, 67100, 66402, 65977, 68959, 66622, 69998,
        66066, 66167, 68162, 70111, 68867, 65654, 65840, 70458, 70458, 67582,
        66109, 66109, 66586, 67250, 68847, 66109, 65897, 67221, 68866, 66592,
        65973, 65973, 68067, 68278, 67129, 66215, 66156, 68803, 67333, 68139,
        65651, 65814, 68115, 67590, 66946, 65814, 66486, 69798, 67487, 69064,
        66460, 66460, 68144, 68670, 69571, 66460, 66035, 69666, 68128, 68597,
        66384, 66384, 70049, 68866, 70172, 65601, 66048, 67403, 67206, 68665,
        66062, 66306, 70049, 67130, 68054, 66306, 66472, 67801, 70160, 70142,
        66185, 66185, 66618, 66618, 67229, 66402, 66048, 69827, 68883, 67963,
        65959, 65959, 68428, 68732, 70404, 66167, 66012, 66928, 68220, 66619,
        65967, 66167, 70616, 67851, 66746, 65654, 65840, 67677, 68064, 68064
]
#audio_token_text="<audio_541><audio_176><audio_1189><audio_3134><audio_1078><audio_413><audio_649><audio_1073><audio_2276><audio_2478><audio_649><audio_71><audio_2594><audio_4291><audio_3274><audio_71><audio_950><audio_2540><audio_3277><audio_1594><audio_109><audio_436><audio_2061><audio_4086><audio_2639><audio_231><audio_950><audio_3267><audio_1367><audio_3956><audio_21><audio_21><audio_1349><audio_2907><audio_2603><audio_218><audio_441><audio_3561><audio_2280><audio_2280><audio_85><audio_1011><audio_2280><audio_2242><audio_2051><audio_77><audio_304><audio_2758><audio_4922><audio_4922><audio_935><audio_825><audio_4922><audio_1909><audio_1990><audio_493><audio_493><audio_4940><audio_3728><audio_3086><audio_845><audio_597><audio_1924><audio_4624><audio_3172><audio_641><audio_681><audio_1133><audio_1539><audio_1143><audio_505><audio_505><audio_1485><audio_1724><audio_1226><audio_911><audio_950><audio_2736><audio_1349><audio_1349><audio_966><audio_966><audio_1764><audio_2342><audio_4624><audio_966><audio_773><audio_2930><audio_1093><audio_2664><audio_748><audio_275><audio_1456><audio_1456><audio_2537><audio_631><audio_441><audio_2537><audio_1404><audio_2528><audio_631><audio_525><audio_2528><audio_1714><audio_1714><audio_304><audio_747><audio_2051><audio_2470><audio_2528><audio_148><audio_148><audio_2493><audio_2639><audio_3267><audio_146><audio_499><audio_1058><audio_1065><audio_1052><audio_259><audio_259><audio_2722><audio_1083><audio_1083><audio_743><audio_950><audio_4576><audio_5040><audio_1025><audio_743><audio_375><audio_1556><audio_3074><audio_5040><audio_375><audio_701><audio_2430><audio_1044><audio_2217><audio_199><audio_199><audio_1594><audio_4785><audio_1741><audio_199><audio_718><audio_1513><audio_3670><audio_1797><audio_800><audio_285><audio_1797><audio_4023><audio_2097><audio_361><audio_361><audio_1056><audio_2427><audio_1371><audio_437><audio_679><audio_1593><audio_3423><audio_4617><audio_679><audio_936><audio_1797><audio_2594><audio_2579><audio_705><audio_705><audio_1410><audio_2592><audio_2409><audio_866><audio_1000><audio_2409><audio_4130><audio_1064><audio_13><audio_293><audio_3645><audio_3810><audio_3274><audio_311><audio_936><audio_1293><audio_3277><audio_3270><audio_830><audio_830><audio_3956><audio_1792><audio_1258><audio_866><audio_893><audio_4971><audio_1097><audio_3279><audio_907><audio_866><audio_4516><audio_1258><audio_1349><audio_18><audio_512><audio_1410><audio_2265><audio_4578><audio_256><audio_256><audio_3264><audio_1143><audio_3882><audio_631><audio_476><audio_1032><audio_1032><audio_2051><audio_431><audio_631><audio_2528><audio_2528><audio_1714><audio_304><audio_304><audio_4922><audio_4922><audio_1077><audio_596><audio_169><audio_4673><audio_4673><audio_1990><audio_81><audio_214><audio_1069><audio_1593><audio_1030><audio_504><audio_504><audio_1030><audio_2540><audio_1125><audio_824><audio_779><audio_1445><audio_1143><audio_1057><audio_411><audio_411><audio_2540><audio_3277><audio_2377><audio_436><audio_436><audio_2315><audio_1293><audio_1816><audio_436><audio_799><audio_2409><audio_2664><audio_1513><audio_843><audio_843><audio_2200><audio_3264><audio_1670><audio_341><audio_341><audio_3868><audio_1217><audio_1725><audio_363><audio_363><audio_1594><audio_2068><audio_3865><audio_363><audio_375><audio_1456><audio_2820><audio_3988><audio_375><audio_936><audio_3311><audio_2430><audio_2036><audio_199><audio_199><audio_2122><audio_4785><audio_1056><audio_624><audio_936><audio_2502><audio_1033><audio_3453><audio_624><audio_433><audio_1826><audio_2633><audio_3267><audio_885><audio_936><audio_1648><audio_3868><audio_1217><audio_930><audio_754><audio_2684><audio_4444><audio_3211><audio_631><audio_304><audio_4088><audio_1146><audio_3232><audio_631><audio_935><audio_1990><audio_4922><audio_1077><audio_304><audio_747><audio_4673><audio_4002><audio_2540><audio_566><audio_824><audio_2718><audio_4023><audio_1445><audio_866><audio_499><audio_1133><audio_5080><audio_1083><audio_705><audio_13><audio_2409><audio_2409><audio_4130><audio_13><audio_936><audio_1064><audio_1083><audio_3645><audio_799><audio_434><audio_2592><audio_5103><audio_4262><audio_434><audio_936><audio_4262><audio_1782><audio_1108><audio_601><audio_601><audio_1966><audio_1518><audio_1399><audio_866><audio_79><audio_1741><audio_2828><audio_1293><audio_1002><audio_1002><audio_2718><audio_2430><audio_1044><audio_866><audio_18><audio_1535><audio_5080><audio_2267><audio_428><audio_799><audio_1353><audio_2684><audio_2341><audio_887><audio_936><audio_4792><audio_4673><audio_3088><audio_591><audio_591><audio_1345><audio_1904><audio_1778><audio_866><audio_133><audio_2097><audio_3900><audio_2712><audio_451><audio_451><audio_5040><audio_1078><audio_1382><audio_866><audio_441><audio_1382><audio_2247><audio_1534><audio_431><audio_631><audio_1534><audio_1404><audio_3331><audio_9><audio_304><audio_2528><audio_2528><audio_1714><audio_935><audio_169><audio_1714><audio_2528><audio_1888><audio_216><audio_214><audio_4361><audio_2337><audio_1229><audio_1003><audio_396><audio_1592><audio_2533><audio_3127><audio_396><audio_950><audio_4624><audio_2430><audio_1044><audio_799><audio_907><audio_4298><audio_4938><audio_1426><audio_907><audio_701><audio_1906><audio_1258><audio_2533><audio_402><audio_402><audio_2399><audio_4792><audio_1052><audio_679><audio_950><audio_4617><audio_2228><audio_3311><audio_679><audio_432><audio_1082><audio_4317><audio_1127><audio_721><audio_132><audio_2370><audio_4785><audio_1966><audio_178><audio_178><audio_2591><audio_4289><audio_1564><audio_866><audio_441><audio_3423><audio_1086><audio_4462><audio_530><audio_631><audio_2626><audio_4575><audio_3331><audio_118><audio_304><audio_4922><audio_4922><audio_2046><audio_573><audio_573><audio_1050><audio_1714><audio_3311><audio_573><audio_361><audio_1685><audio_3330><audio_1056><audio_437><audio_437><audio_2531><audio_2742><audio_1593><audio_679><audio_620><audio_3267><audio_1797><audio_2603><audio_115><audio_278><audio_2579><audio_2054><audio_1410><audio_278><audio_950><audio_4262><audio_1951><audio_3528><audio_924><audio_924><audio_2608><audio_3134><audio_4035><audio_924><audio_499><audio_4130><audio_2592><audio_3061><audio_848><audio_848><audio_4513><audio_3330><audio_4636><audio_65><audio_512><audio_1867><audio_1670><audio_3129><audio_526><audio_770><audio_4513><audio_1594><audio_2518><audio_770><audio_936><audio_2265><audio_4624><audio_4606><audio_649><audio_649><audio_1082><audio_1082><audio_1693><audio_866><audio_512><audio_4291><audio_3347><audio_2427><audio_423><audio_423><audio_2892><audio_3196><audio_4868><audio_631><audio_476><audio_1392><audio_2684><audio_1083><audio_431><audio_631><audio_5080><audio_2315><audio_1210><audio_118><audio_304><audio_2141><audio_2528><audio_2528>"

In [ ]:
audio_token_text = tts_engine.tokenizer.decode(audio_token_ids)
print(audio_token_text)

In [ ]:
import torch
# audio vq codes
tensor_audio_token_ids = torch.tensor([audio_token_ids]).to(torch.long).to("cuda")-65536
print(tensor_audio_token_ids)

In [ ]:
tts_speech = tts_engine.common_cosy_model.token_to_wav_offline(
    tensor_audio_token_ids,
    prompt_speaker_info["cosy_speech_feat"].to(torch.bfloat16),
    prompt_speaker_info["cosy_speech_feat_len"],
    prompt_speaker_info["cosy_prompt_token"],
    prompt_speaker_info["cosy_prompt_token_len"],
    prompt_speaker_info["cosy_speech_embedding"].to(torch.bfloat16),
)

In [ ]:
print(tts_speech,tts_speech.shape)

In [ ]:
import torchaudio
torchaudio.save(f"/content/output_tts.wav", tts_speech, 22050)

In [ ]:
from IPython.display import Audio
Audio('/content/output_tts.wav',autoplay=True)

## flow (CFM) generate mel spec from audio vq codes

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
tts_mel = tts_engine.common_cosy_model.model.flow.inference(
    token=tensor_audio_token_ids.to(device),
    token_len=torch.tensor([tensor_audio_token_ids.size(1)], dtype=torch.int32).to(device),
    prompt_token=prompt_speaker_info["cosy_prompt_token"].to(device),
    prompt_token_len=prompt_speaker_info["cosy_prompt_token_len"].to(device),
    prompt_feat=prompt_speaker_info["cosy_speech_feat"].to(torch.bfloat16).to(device),
    prompt_feat_len=prompt_speaker_info["cosy_speech_feat_len"].to(device),
    embedding=prompt_speaker_info["cosy_speech_embedding"].to(torch.bfloat16).to(device),
)

In [ ]:
print(tts_mel)

## vocoder(hifi) generate waveform from mel spec

In [ ]:
tts_speech_waveform = tts_engine.common_cosy_model.model.hift.inference(mel=tts_mel.float())[0].cpu()

In [ ]:
print(tts_speech_waveform)

In [ ]:
import torchaudio
torchaudio.save(f"/content/output_tts_again.wav", tts_speech_waveform, 22050)

In [ ]:
from IPython.display import Audio
Audio('/content/output_tts_again.wav',autoplay=True)

## batch generate waveform from aduio vq codes

In [ ]:
print(tensor_audio_token_ids,tensor_audio_token_ids.shape)

In [ ]:
# prompt: 将tensor_audio_token_ids 按 batch_size 批量 生成

import torch

# Assuming tensor_audio_token_ids is already defined as in your original code
# and has shape (1, sequence_length)

def batch_generate(tensor_audio_token_ids, batch_size):
  """
  Generates batches of audio token IDs.

  Args:
    tensor_audio_token_ids: A tensor of audio token IDs.
    batch_size: The desired batch size.

  Returns:
    A list of tensors, where each tensor is a batch of audio token IDs.
  """
  sequence_length = tensor_audio_token_ids.shape[1]
  num_batches = (sequence_length + batch_size - 1) // batch_size
  batched_ids = []

  for i in range(num_batches):
      start_index = i * batch_size
      end_index = min((i + 1) * batch_size, sequence_length)
      batch = tensor_audio_token_ids[:, start_index:end_index]
      batched_ids.append(batch)

  return batched_ids

In [ ]:
import math
#batch_size = 30
stream_factor=2 # >=2 increase for better speech quality, but rft slow (speech quality vs rft slow)
batch_size = math.ceil(stream_factor*tts_engine.common_cosy_model.model.flow.input_frame_rate)
print(batch_size)
batched_tensor_audio_token_ids = batch_generate(tensor_audio_token_ids, batch_size)
for batched_tensor_audio_token_id in batched_tensor_audio_token_ids:
  print(batched_tensor_audio_token_id,batched_tensor_audio_token_id.shape)
print(len(batched_tensor_audio_token_ids))

In [ ]:
sub_tts_speechs = []
# Now you can iterate through the batched_tensor_audio_token_ids and process each batch
for batch in batched_tensor_audio_token_ids:
    print(batch.shape) # Print the shape of each batch
    # Process each batch with tts_engine.common_cosy_model.token_to_wav_offline()
    sub_tts_speech = tts_engine.common_cosy_model.token_to_wav_offline(
        batch,
        prompt_speaker_info["cosy_speech_feat"].to(torch.bfloat16),  # Pass the correct input for this function
        prompt_speaker_info["cosy_speech_feat_len"],
        prompt_speaker_info["cosy_prompt_token"],
        prompt_speaker_info["cosy_prompt_token_len"],
        prompt_speaker_info["cosy_speech_embedding"].to(torch.bfloat16),
    )
    print(sub_tts_speech.shape)
    sub_tts_speechs.append(sub_tts_speech)


In [ ]:
print(len(sub_tts_speechs))

In [ ]:
# prompt: 将 sub_tts_speechs 合并成 一个 tensor

import torch

# Assuming sub_tts_speechs is a list of tensors
# and all tensors in the list have the same number of channels
# but possibly different lengths.

def merge_tensors(sub_tts_speechs):
  """Merges a list of tensors into a single tensor.

  Args:
    sub_tts_speechs: A list of tensors.

  Returns:
    A single tensor with all the sub tensors concatenated along the time dimension.
    Returns None if the input list is empty or contains tensors with inconsistent shapes.
  """
  if not sub_tts_speechs:
    return None

  num_channels = sub_tts_speechs[0].shape[0]
  total_length = sum(tensor.shape[1] for tensor in sub_tts_speechs)
  merged_tensor = torch.empty(num_channels, total_length, dtype=sub_tts_speechs[0].dtype, device=sub_tts_speechs[0].device)
  current_position = 0

  for tensor in sub_tts_speechs:
      if tensor.shape[0] != num_channels:
          print("Error: Tensors have inconsistent number of channels.")
          return None

      merged_tensor[:, current_position:current_position + tensor.shape[1]] = tensor
      current_position += tensor.shape[1]

  return merged_tensor




In [ ]:
merged_speech = merge_tensors(sub_tts_speechs)
print(merged_speech)

if merged_speech is not None:
    print(merged_speech.shape)
    torchaudio.save(f"/content/merged_output.wav", merged_speech, 22050)
    print("Merged audio saved to /content/merged_output.wav")
else:
    print("Could not merge audio tensors.")

In [ ]:
from IPython.display import Audio
Audio('/content/merged_output.wav',autoplay=True)